# Explore hologram data  Repeatability on Ozone

- author Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date 2024-10-09
- last update : 2024-10-12 : Flag on collimator on
- Kernel @usdf **w_2024_37**
- Office emac : mamba_py311
- Home emac : base (conda)
- laptop : conda_py310

**Goal** : Show Night variations of Ozone wrt date and Time. Fit a straight line.

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figsOzoneRepeatability"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove this if want to run faster the ntoebook
# import ipywidgets as widgets
# %matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

## Configuration

In [ ]:
FLAG_WITHCOLLIMATOR = True

In [ ]:
version_results = "v4"
legendtag = {"v1" : "old v3.1.0",
            "v2" : "v3.1.0-PWV<10mm",
            "v3" : "v3.1.0-PWV<15mm",
            "v4" : "v3.1.0-extended"}

In [ ]:
atmfilenamesdict = {"v1" : "data/spectro/auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_testWithMaskedEdges_newBoundaries_newPolysRescaled_newFitBounds_adjustA1_lockedOrder2_removeThroughputTails_2.npy",
                    "v2" : "auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_PeekFinder.npy",
                    "v3" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z.npy",
                    "v4" : "u_dagoret_auxtel_atmosphere_202301_v3.1.0_doSensorFlat_rebin2_lockedOrder2_FixA1_FixA2_FitAngstrom_WithGaia_freePressure_newThroughput6_BG40Scaled1.09_AtmoFitPressureA2_SpecErr_PeekFinder_20240924T161119Z_spectrfullextend.npy"}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> 20230930]

#### Series on spec

In [ ]:
ser_spec_size = df_spec.groupby(["nightObs"]).size()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(ser_spec_size.values,bins=50,facecolor="b")
ax.set_title("nb obs per night")
ax.set_xlabel("Nobs/night")

##### Make 3 series

In [ ]:
ser_CHI2_FIT = df_spec[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_O3 = df_spec[["ozone [db]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_O3_CHI2_FIT = df_spec[["ozone [db]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
#ser_O3

In [ ]:
#ser_CHI2_FIT

In [ ]:
#ser_O3_CHI2_FIT 

### Plot Ozone and Chi2 before anly selection

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean Ozone per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median Ozone per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3.unstack()["ozone [db]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for Ozone per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="r",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night")
plt.tight_layout()

In [ ]:
#ser_O3_CHI2_FIT 

In [ ]:
ser_O3_CHI2_FIT.loc[20231010,:]

In [ ]:
df_unstack = ser_O3_CHI2_FIT.loc[20231010,:].unstack()
df_unstack

In [ ]:
ser_O3_CHI2_FIT.loc[20231010,"ozone [db]_x"]["count"]

In [ ]:
df_unstack.loc["ozone [db]_x","count"]

#### Add aggregate data added to pandas dataframe

In [ ]:
def FillAgreggates(row):
    the_nightObs = row["nightObs"]
    df_night = ser_O3_CHI2_FIT.loc[the_nightObs,:].unstack()
    count = df_night.loc["ozone [db]_x","count"]
    o3min = df_night.loc["ozone [db]_x","min"]
    o3max = df_night.loc["ozone [db]_x","max"]
    o3mean = df_night.loc["ozone [db]_x","mean"]
    o3median = df_night.loc["ozone [db]_x","median"]
    o3std = df_night.loc["ozone [db]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_o3min":o3min,"_o3max":o3max,"_o3mean":o3mean,"_o3median":o3median,"_o3std":o3std,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
row = df_spec.iloc[0]

In [ ]:
row

In [ ]:
FillAgreggates(row)

#### join dataframe + aggregates

In [ ]:
df_spec = df_spec.join(df_spec.apply(FillAgreggates,axis=1,result_type="expand"))

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

### Add the Time in pd.datetime

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()
df_spec.plot(x="Time",y="ozone [db]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("ozone [db]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Ozone meas by holo vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

## Apply Quality selection cuts

In [ ]:
def getSelectionCut(df_spec, chi2max=20., o3min=1.0, o3max = 599.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["ozone [db]_x"] > o3min) & (df_spec["ozone [db]_x"] < o3max) 
    return cut

In [ ]:
cut = getSelectionCut(df_spec)

In [ ]:
df_spec_sel = df_spec[cut].drop(labels=['_count', '_o3min', '_o3max', '_o3mean', '_o3median', '_o3std','_chi2min', '_chi2max', '_chi2mean', '_chi2median', '_chi2std'],axis=1)

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)

## Compute per-night aggregates

### Compute series per night

In [ ]:
ser_O3_CHI2_FIT_sel = df_spec_sel[["ozone [db]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
def FillAgreggatesSel(row):
    the_nightObs = row["nightObs"]
    df_night = ser_O3_CHI2_FIT_sel.loc[the_nightObs,:].unstack()
    count = df_night.loc["ozone [db]_x","count"]
    o3min = df_night.loc["ozone [db]_x","min"]
    o3max = df_night.loc["ozone [db]_x","max"]
    o3mean = df_night.loc["ozone [db]_x","mean"]
    o3median = df_night.loc["ozone [db]_x","median"]
    o3std = df_night.loc["ozone [db]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_o3min":o3min,"_o3max":o3max,"_o3mean":o3mean,"_o3median":o3median,"_o3std":o3std,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
df_spec_sel = df_spec_sel.join(df_spec_sel.apply(FillAgreggatesSel,axis=1,result_type="expand"))

## Recompute night boundaries

In [ ]:
dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

df_spec_sel.plot(x="Time",y="ozone [db]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg,ms=10)
ax.set_ylabel("ozone [db]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Ozone measured by holo selected vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()
figname =f"{pathfigs}/ozone_allpoints_allnights"+figtype
plt.savefig(figname)
plt.show()

## Plot series on selected data

In [ ]:
ser_CHI2_FIT_sel = df_spec_sel[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_O3_sel = df_spec_sel[["ozone [db]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean Ozone per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median Ozone per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_O3_sel.unstack()["ozone [db]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for Ozone per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night after selection")
plt.tight_layout()

## Show time variation each night

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_o3_curve):
    """
    """

    x = df_night_o3_curve["dt"].values
    y = df_night_o3_curve["ozone [db]_x"].values
    yerr = df_night_o3_curve["ozone [db]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

### Plot night by night

In [ ]:
df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == 20240923]
#select the variables
df_night_o3_curve = df_spec_night[["nightObs","Time","ozone [db]_x","ozone [db]_err_x","ozone [db]_y","ozone [db]_err_y","TARGET"]]
tmin = df_night_o3_curve["Time"].min()
df_night_o3_curve["dt"] = (df_night_o3_curve["Time"] - tmin).dt.total_seconds()/3600.

In [ ]:
def ComputeRepeatability(df):
    """
    Compute Repeatability of Ozone for Spectrogram and and Spectrum
    
    """
    N = len(df)
    dfout = pd.DataFrame(index=df.index,columns = ["nightObs","dt","dt_rep","dOZx_rep","dOZy_rep","targflag_rep","Npoints"])
    dfout["targflag_rep"].astype(bool)
    #dfout["Npoints"].astype(int)
    
    target_old = "No"
    time_old = 0.
    OZx_old = 0.
    OZy_old = 0.
    
    for index in range(N):
        
        nightObs =  df.iloc[index]["nightObs"]
        
        if index ==0:
            dt0 = df.iloc[index]["dt"]
            dfout.iloc[index] = [ nightObs,dt0, 0., 0., 0., False,N]
        else:
            target_new = df.iloc[index]["TARGET"]
            time_new = df.iloc[index]["dt"]
            OZx_new = df.iloc[index]["ozone [db]_x"]
            OZy_new = df.iloc[index]["ozone [db]_y"]
            
            flag_target = (target_new == target_old)
            dOZx_rep = OZx_new - OZx_old
            dOZy_rep = OZy_new - OZy_old
            dt_rep = (time_new-time_old)*3600. # in seconds
            
            dfout.iloc[index] = [ nightObs,time_new, dt_rep, dOZx_rep, dOZy_rep, flag_target,N]
        
        target_old = df.iloc[index]["TARGET"]
        time_old = df.iloc[index]["dt"]
        OZx_old = df.iloc[index]["ozone [db]_x"]
        OZy_old = df.iloc[index]["ozone [db]_y"]
        
    return dfout

In [ ]:
ComputeRepeatability(df_night_o3_curve).head()

In [ ]:
all_df_repeatability = []

# loop on nights
for night in all_selected_nights:
    # Select the night
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    
    # Select the variables
    df_night_o3_curve = df_spec_night[["nightObs","Time","ozone [db]_x","ozone [db]_err_x","ozone [db]_y","ozone [db]_err_y","TARGET"]]

    tmin = df_night_o3_curve["Time"].min()

    # Convert in hours
    df_night_o3_curve["dt"] = (df_night_o3_curve["Time"] - tmin).dt.total_seconds()/3600.
    
    # Compute the repeatability on PWV x and y
    df_rep = ComputeRepeatability(df_night_o3_curve)
    
    # Keep all repeatability
    all_df_repeatability.append(df_rep)
    
    cut_on_dt = (df_rep["dt_rep"]>0.) & (df_rep["dt_rep"]< 120.)   
    cut_on_target = df_rep["targflag_rep"]
    
    cut_loose = cut_on_dt
    cut_strong = cut_on_dt & cut_on_target 
        
    # plot  
    fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(12,4))
    
    leg1 = ax1.get_legend()
    leg2 = ax2.get_legend()
    
      
    df_rep[cut_strong]["dOZx_rep"].hist(ax=ax1,bins=60,range=(-50,50),facecolor="b",legend=leg1,label="Same target")
    df_rep[cut_strong]["dOZy_rep"].hist(ax=ax2,bins=60,range=(-50,50),facecolor="b")
    
    df_rep[cut_loose]["dOZx_rep"].hist(ax=ax1,bins=60,range=(-50,50),color="r",lw=3,histtype="step")
    df_rep[cut_loose]["dOZy_rep"].hist(ax=ax2,bins=60,range=(-50,50),color="r",lw=3,histtype="step")
    
    ax1.set_xlabel("Ozone (DU)")
    ax1.set_title("Spectrogram")
    ax2.set_xlabel("Ozone (DU)")
    ax2.set_title("1D-Spectrum")
    
    df_rep[cut_strong].plot.scatter(x="dOZx_rep",y="dOZy_rep",marker='.',c="b",ax=ax3)
    ax3.set_xlim(-50.,50.)
    ax3.set_ylim(-50.,50.)
    ax3.grid()
    ax3.set_xlabel("Ozone (2D) (DU)")
    ax3.set_ylabel("Ozone (1D) (DU)")
    
    
    title = f"Ozone repeatability for Night {night}"
    plt.suptitle(title) 
    plt.tight_layout()
    #figname =f"{pathfigs}/pwv_per_night_{night}"+figtype
    #    plt.savefig(figname)
    plt.show()

## Merge all repeatability dataFrame

In [ ]:
df_allrep = pd.concat(all_df_repeatability)

In [ ]:
df_allrep.head()

## Selection

- stat on strong cut

In [ ]:
cut_on_dt = (df_allrep["dt_rep"]>0.) & (df_allrep["dt_rep"]< 120.) & (df_allrep["Npoints"] > 10.)     
cut_on_target = df_allrep["targflag_rep"]
    
cut_loose = cut_on_dt
cut_strong = cut_on_dt & cut_on_target 

In [ ]:
dfl = df_allrep[cut_loose][["dt","dt_rep","dOZx_rep","dOZy_rep"]]
dfs = df_allrep[cut_strong][["dt","dt_rep","dOZx_rep","dOZy_rep"]]

In [ ]:
dfs_stat = dfs.aggregate(["count","mean","std"])
meanx = dfs_stat.loc["mean","dOZx_rep"]
stdx = dfs_stat.loc["std","dOZx_rep"]
meany = dfs_stat.loc["mean","dOZy_rep"]
stdy = dfs_stat.loc["std","dOZy_rep"]

dfs["ddOZx_rep"] = np.abs(dfs["dOZx_rep"] - meanx)
dfs["ddOZy_rep"] = np.abs(dfs["dOZy_rep"] - meany)

dfs_stat = dfs.aggregate(["count","mean","std"])

meanx = dfs_stat.loc["mean","dOZx_rep"]
stdx = dfs_stat.loc["std","dOZx_rep"]
madx =  dfs_stat.loc["mean","ddOZx_rep"]
sigma_madx = np.sqrt(np.pi/2.)*madx

meany = dfs_stat.loc["mean","dOZy_rep"]
stdy = dfs_stat.loc["std","dOZy_rep"]
mady =  dfs_stat.loc["mean","ddOZy_rep"]
sigma_mady = np.sqrt(np.pi/2.)*mady

textstr_ozx = "$\delta_{oz}$ = "+ f"{meanx:.1f}" + "$\pm + $" + f"{stdx:.2f} DU" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_madx:.2f} DU"
textstr_ozy = "$\delta_{oz}$ = "+ f"{meany:.1f}" + "$\pm + $" + f"{stdy:.2f} DU" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_mady:.2f} DU"

### Plot repeatability

In [ ]:
# plot  
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(12,4))
    
leg1 = ax1.get_legend()
leg2 = ax2.get_legend()
    
      
df_allrep[cut_strong]["dOZx_rep"].hist(ax=ax1,bins=60,range=(-100,100),facecolor="b",legend=leg1,label="Same target")
df_allrep[cut_strong]["dOZy_rep"].hist(ax=ax2,bins=60,range=(-100,100),facecolor="b")
    
df_allrep[cut_loose]["dOZx_rep"].hist(ax=ax1,bins=60,range=(-100.,100.),color="r",lw=3,histtype="step")
df_allrep[cut_loose]["dOZy_rep"].hist(ax=ax2,bins=60,range=(-100.,100.),color="r",lw=3,histtype="step")
    
ax1.set_xlabel("$\delta$ Ozone (DU)")
ax1.set_title("Spectrogram")
ax2.set_xlabel("$\delta$ Ozone (DU)")
ax2.set_title("1D-Spectrum")

ax1.text(0.05, 0.95, textstr_ozx, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.text(0.05, 0.95, textstr_ozy, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
    
df_allrep[cut_strong].plot.scatter(x="dOZx_rep",y="dOZy_rep",marker='.',c="b",ax=ax3)
ax3.set_xlim(-100,100.)
ax3.set_ylim(-100,100.)
ax3.grid()
ax3.set_xlabel("$\delta $ Ozone (2D) (DU)")
ax3.set_ylabel("$\delta $ Ozone (1D) (DU)")

if FLAG_WITHCOLLIMATOR:
    title = "Ozone repeatability over all nights (10/2023-09/2024)"
else:
    title = "Ozone repeatability over all nights (01/2023-09/2024)"


plt.suptitle(title)
plt.tight_layout()